In [1]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
dir_cmc='F:/data/sst/cmc/CMC0.2deg/v2/'
dir_cmc_clim='F:/data/sst/cmc/CMC0.2deg/v2/climatology/'

def get_filename(lyr,idyjl):
    podaac_dir_v3 = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.1deg/v3/'
    podaac_dir_v2 = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/'
    d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    sdym=str(d.day).zfill(2)
    sjdy=str(idyjl).zfill(3)
    if lyr<2017:
        cmc_filename=podaac_dir_v2 + syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc'
    else:
        cmc_filename=podaac_dir_v3 + syr + smon + sdym + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.1deg-GLOB-v02.0-fv03.0.nc'
    

In [4]:
#testing 0.1 CMC
filename = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/1994/002/19940102120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])
ds2 = ds.interp(lat = new_lat,lon = new_lon)

#for 2017 - present use 0.1 CMC interpolated onto 0.2 grid for monthly averages and run on pythonanywhere
for lyr in range(2019,2021): #2017):
#    ds_mnth=[]
#    for imon in range(1,13):
#        init = 0
    for idyjl in range(1,366):
        cmc_filename = get_filename(lyr,idyjl)
    #cmc_filename = podaac_dir + syr + '/' + sjdy + '/' + fname_tem
        #print(cmc_filename)
        ds = xr.open_dataset(cmc_filename,drop_variables=['analysis_error','sea_ice_fraction'])           
        ds_masked = ds.where(ds['mask'] == 1.) 
        ds.close()
        ds_masked['sq_sst']=ds_masked.analysed_sst**2
        if init==0:
            ds_sum = ds_masked
            init = 1
        else:
            ds_sum = xr.concat([ds_sum,ds_masked],dim = 'time')
        print(idyjl,ds_sum.dims)
#        ds_sum = ds_sum.mean('time',skipna=True)
#        ds_mnth.append(ds_sum)
    ds_sum = ds_sum.mean('time')
#    combined = ds_sum.resample(time='1M').mean()
#    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='monthly_average_' + syr + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
    cmc_filename_out = './data/' + fname_tem
#    combined.to_netcdf(cmc_filename_out)
    ds_sum.to_netcdf(cmc_filename_out)


NameError: name 'ds' is not defined